# 📰 Fake News Classfication

**Data Info**
we are going to take the following approach:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modelling
6. Experimentation


## 1. Problem definition

Given a fake news , can we able to classify it as fake or not ?


## 2. Data

**information on Train csv**

  `id`: unique id for a news article

  `title`: the title of a news article

  `author`: author of the news article

  `text`: the text of the article; could be incomplete

  `label`: a label that marks the article as potentially unreliable

  1: unreliable

  0: reliable

`test.csv`: A testing training dataset with all the same attributes at train.csv without the label.







In [1]:
!kaggle competitions download -c fake-news

 99%|█████████████████████████████████████▌| 46.0M/46.5M [00:10<00:00, 4.92MB/s]
100%|██████████████████████████████████████| 46.5M/46.5M [00:10<00:00, 4.50MB/s]


In [2]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [3]:
import pandas as pd

import numpy as np

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords # [use to remove unwanted word from the sentence]

from nltk.stem.porter import PorterStemmer

from sklearn.preprocessing import OneHotEncoder

# Load modal
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

# Load evaluation metrics

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
import re

In [4]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/magesh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
train_df=pd.read_csv("train.csv")

In [6]:
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
train_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
# fill null values
train_df.fillna("",axis=1,inplace=True)

In [17]:
train_df['content']=train_df['author'] + " " + train_df['title']

In [11]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/magesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
def stemming(content:str)->str:
  """
  this little function helps us to remove the unwanted words from the content
  1.we will remove the numerical char from the conent
  2. use stop_word to remove unwanted word
  3. use stemming process and return the word
  """

  port_stemm=PorterStemmer()

  stem_content=re.sub('[^a-zA-Z]',' ',content) # replace all the numerical
  stem_content=stem_content.lower()
  # split it
  stem_content=stem_content.split()

  # apply stopword and stemmer
  stem_content=[port_stemm.stem(word) for word in stem_content if not word in stopwords.words('english')]

  return ' '.join(stem_content)
 


In [18]:
print(">> Before Stemming process",train_df['content'][0])


print(">> After stemming process",stemming(train_df['content'][0]))

>> Before Stemming process Darrell Lucus House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It
>> After stemming process darrel lucu hous dem aid even see comey letter jason chaffetz tweet


In [19]:
# apply stemming process to all the content

train_df['content']=train_df['content'].apply(stemming)

In [20]:
# create x & y
X=train_df['content'].values# Features
y=train_df['label'].values # target


In [21]:

vectorizer=TfidfVectorizer()

vectorizer.fit(X) # fit the feature to the vector model

X=vectorizer.transform(X)

In [22]:
X.shape

(20800, 17128)

In [26]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.4154496266472162
  (2, 9620)	0.4935149294364995
  (2, 5968)	0.34746133867282925
  (2, 5389)	0.38665305511826153
  (2, 3103)	0.4609748958322965
  :	:
  (20797, 13122)	0.24825263521976057
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396505
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.17455348025522197
  (20797, 9518)	0.2954

In [23]:

# train -> 80%
# test -> 20%
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y) # split equal class size

In [30]:
x_train

<16640x17128 sparse matrix of type '<class 'numpy.float64'>'
	with 168499 stored elements in Compressed Sparse Row format>

In [31]:
clf=RandomForestClassifier()

In [35]:
y_train.shape

(16640,)

In [36]:
clf.fit(X,y)

ValueError: Unknown label type: 'unknown'